In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import utils.data as data

train, test = data.get_train_test_split(0.01, 42)

In [3]:
dataset = data.Marconi100Dataset(test)
df, lab = dataset[0]

100%|██████████| 3/3 [00:04<00:00,  1.34s/it]


In [4]:
lab.unique()

array([0, 1])

In [5]:
df.shape

(13195, 460)

In [6]:
import torch

tensor = torch.ones(df.shape)
print(tensor.shape)

torch.Size([13195, 460])


In [10]:
import numpy as np

def generate_masks(M):
    masks = []
    for i in range(0, M):
        x = tensor.clone().detach()

        #get column indices for the features to mask
        col_idxs = np.random.choice(range(0, df.shape[1]), 10, replace=False)
        #get row indices for the samples to mask
        row_idxs = np.random.choice(range(0, df.shape[0]), 10, replace=False)
        if i == 0:
            print("col_idxs: ", col_idxs)
            print("row_idxs: ", row_idxs)
            for i in row_idxs:
                for j in col_idxs:
                    x[i][j] = 0

        mask = x > 0
        masks.append(mask)
    return masks

def generate_mask_V1(shape, n):
    
    # inefficiente
    n_mask = int(np.prod(shape) / n)
    
    mask = torch.ones(shape)
    print(np.count_nonzero(mask == 0), n_mask)
    
    while np.count_nonzero(mask == 0) < n_mask:
        
        # get column index for the element to mask
        col_idx = np.random.choice(range(shape[1]), replace=False)
        # get row index for the element to mask
        row_idx = np.random.choice(range(shape[0]), replace=False)
        mask[row_idx, col_idx] = 0
        
    return mask

def generate_mask_V2(shape, n):
    
    # molto più efficiente
    
    mask = torch.ones(np.prod(shape))
    
    n_mask = int(np.prod(shape) / n)
    
    # make sure as many as n_mask samples are masked
    mask[:n_mask] = 0
    
    mask = torch.reshape(mask, shape)
    
    # permute rows
    mask = np.random.permutation(mask)
    
    # permute columns
    mask = mask[:, np.random.permutation(mask.shape[1])]
    
    return mask

def generate_mask_V3(shape, n):
    
    # un po' più efficiente
    mask = np.ones(np.prod(shape))
    n_mask = int(np.prod(shape) / n)
    # make sure as many as n_mask samples are masked
    mask[:n_mask] = 0

    # permute elements
    mask = np.random.permutation(mask)

    # reshape to the input shape
    mask = torch.tensor(mask).reshape(shape)
    
    return mask

In [15]:
generate_mask_V3((5, 5), 2)

tensor([[1., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 1.]], dtype=torch.float64)

In [ ]:
def generate_non_overlapping_masks(shape, n):
    mask = np.ones(np.prod(shape))
    n_mask = int(np.prod(shape) / n)
    # make sure as many as n_mask samples are masked
    mask[:n_mask] = 0

    # permute elements
    mask = np.random.permutation(mask)

    # reshape to the input shape
    mask = torch.tensor(mask).reshape(shape)
    
    return mask

In [8]:
res = tensor.unfold(0, 1024, 512)
print(res.shape)

torch.Size([24, 460, 1024])


In [111]:
masks = generate_mask_V1(tensor.shape, 2000)
# first_mask = masks[0]
# print indices of holes
# print(torch.logical_not(first_mask).nonzero())

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 11.4 µs
0 3078


In [115]:
mask = generate_mask_V2(tensor.shape, 2000)
print(np.count_nonzero(mask == 0))
print(np.count_nonzero(mask == 0) + np.count_nonzero(mask) == np.prod(mask.shape))
# print indices of holes
# print(torch.logical_not(mask).nonzero())

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 15.7 µs
tensor([1., 1., 1.,  ..., 1., 1., 1.])
tensor([0., 0., 0.,  ..., 1., 1., 1.])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])
3078
True


In [2]:
from scikitimage.util.shape import view_as_windows

ModuleNotFoundError: No module named 'scikitimage'

In [26]:
arr = np.ones((25, 25))
sample = np.random.choice(len(arr), 5)
print(sample.shape)
print(sample)

(5,)
[19 15 16  8 24]


In [15]:
import deep_fib.data as data_dfib
import numpy as np

dataset = data_dfib.PointMaskDataset(test, 10)

df, mask, lab = dataset[0]

AttributeError: module 'deep_fib.data' has no attribute 'PointMaskDataset'

In [15]:
int(np.prod(df.shape) / dataset.n)

606970

In [16]:
(mask == -1).sum()

tensor(577489)

In [17]:
(mask != -1).sum()

tensor(5492211)